In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/content/arabic-empathetic-conversations.csv')

In [3]:
df.head()

,emotion,context,response
0,sentimental,أتذكر أنني ذهبت لمشاهدة الألعاب النارية مع أعز...,هل كان هذا صديقًا كنت تحبه أم مجرد أفضل صديق؟
1,sentimental,كان هذا أفضل صديق. اشتقت لها.,اين ذهبت؟
2,sentimental,لم نعد نتحدث.,هل كان هذا شيء حدث بسبب جدال؟
3,afraid,أشعر وكأنني ضرب على جدار فارغ عندما أرى الظلام,أجل؟ أنا حقا لا أرى كيف
4,afraid,ألا تشعر بذلك .. إنه لأمر عجيب,أصطدم في الواقع بجدران فارغة في كثير من الأحيا...


In [4]:
df.shape

(36628, 3)

In [5]:
df = df.drop('emotion', axis=1)
df.head()

,context,response
0,أتذكر أنني ذهبت لمشاهدة الألعاب النارية مع أعز...,هل كان هذا صديقًا كنت تحبه أم مجرد أفضل صديق؟
1,كان هذا أفضل صديق. اشتقت لها.,اين ذهبت؟
2,لم نعد نتحدث.,هل كان هذا شيء حدث بسبب جدال؟
3,أشعر وكأنني ضرب على جدار فارغ عندما أرى الظلام,أجل؟ أنا حقا لا أرى كيف
4,ألا تشعر بذلك .. إنه لأمر عجيب,أصطدم في الواقع بجدران فارغة في كثير من الأحيا...


In [7]:
df = df.rename(columns={
    'context': 'Question',
    'response': 'Answer',
})

df.head()

,Question,Answer
0,أتذكر أنني ذهبت لمشاهدة الألعاب النارية مع أعز...,هل كان هذا صديقًا كنت تحبه أم مجرد أفضل صديق؟
1,كان هذا أفضل صديق. اشتقت لها.,اين ذهبت؟
2,لم نعد نتحدث.,هل كان هذا شيء حدث بسبب جدال؟
3,أشعر وكأنني ضرب على جدار فارغ عندما أرى الظلام,أجل؟ أنا حقا لا أرى كيف
4,ألا تشعر بذلك .. إنه لأمر عجيب,أصطدم في الواقع بجدران فارغة في كثير من الأحيا...


In [9]:
import re
def clean_text(text):
  """
  Removes punctuation, numbers, and dates from the text.
  """
  text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
  text = re.sub(r'\d+', '', text)  # Remove numbers
  text = re.sub(r'\b\d{1,2}/\d{1,2}/\d{2,4}\b', '', text)  # Remove dates (DD/MM/YYYY)
  return text


# Clean questions and answers
df['Question'] = df['Question'].astype(str).apply(clean_text)
df['Answer'] = df['Answer'].astype(str).apply(clean_text)

df.head()


,Question,Answer
0,أتذكر أنني ذهبت لمشاهدة الألعاب النارية مع أعز...,هل كان هذا صديقا كنت تحبه أم مجرد أفضل صديق
1,كان هذا أفضل صديق اشتقت لها,اين ذهبت
2,لم نعد نتحدث,هل كان هذا شيء حدث بسبب جدال
3,أشعر وكأنني ضرب على جدار فارغ عندما أرى الظلام,أجل أنا حقا لا أرى كيف
4,ألا تشعر بذلك إنه لأمر عجيب,أصطدم في الواقع بجدران فارغة في كثير من الأحيا...


In [10]:
from sklearn.model_selection import train_test_split

# تقسيم البيانات إلى تدريب واختبار
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# تحويل البيانات إلى قوائم
train_questions = train_df['Question'].tolist()
train_answers = train_df['Answer'].tolist()

test_questions = test_df['Question'].tolist()
test_answers = test_df['Answer'].tolist()

In [11]:
import torch
from transformers import MT5Tokenizer, MT5ForConditionalGeneration
from torch.utils.data import DataLoader, Dataset

# تحديد الجهاز
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# تحميل النموذج المسبق التدريب
model_name = 'csebuetnlp/mT5_multilingual_XLSum'
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

# تحضير البيانات
train_encodings = tokenizer(train_questions, truncation=True, padding=True, return_tensors="pt")
test_encodings = tokenizer(test_questions, truncation=True, padding=True, return_tensors="pt")

# إضافة الإجابات كـ 'labels'
train_encodings['labels'] = tokenizer(train_answers, truncation=True, padding=True, return_tensors="pt").input_ids
test_encodings['labels'] = tokenizer(test_answers, truncation=True, padding=True, return_tensors="pt").input_ids

# فئة Dataset
class QADataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = QADataset(train_encodings)
test_dataset = QADataset(test_encodings)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [13]:
import time

# التدريب
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

for epoch in range(2):  # عدد العهود
    model.train()
    total_loss = 0
    for batch_idx, batch in enumerate(train_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if batch_idx % 10 == 0:
            print(f"Epoch {epoch + 1}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item()}")

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Average Loss: {avg_loss}")

Epoch 1, Batch 0/3663, Loss: 0.7074000239372253
Epoch 1, Batch 10/3663, Loss: 0.6158146858215332
Epoch 1, Batch 20/3663, Loss: 0.6094129681587219
Epoch 1, Batch 30/3663, Loss: 0.5665369033813477
Epoch 1, Batch 40/3663, Loss: 0.4211485981941223
Epoch 1, Batch 50/3663, Loss: 0.6198292970657349
Epoch 1, Batch 60/3663, Loss: 0.588981032371521
Epoch 1, Batch 70/3663, Loss: 0.4179645776748657
Epoch 1, Batch 80/3663, Loss: 0.5001742243766785
Epoch 1, Batch 90/3663, Loss: 0.33428847789764404
Epoch 1, Batch 100/3663, Loss: 0.5983431935310364
Epoch 1, Batch 110/3663, Loss: 0.6263098120689392
Epoch 1, Batch 120/3663, Loss: 0.4018920958042145
Epoch 1, Batch 130/3663, Loss: 0.4050738513469696
Epoch 1, Batch 140/3663, Loss: 0.30030786991119385
Epoch 1, Batch 150/3663, Loss: 0.480148583650589
Epoch 1, Batch 160/3663, Loss: 0.31187912821769714
Epoch 1, Batch 170/3663, Loss: 0.45838242769241333
Epoch 1, Batch 180/3663, Loss: 0.43068376183509827
Epoch 1, Batch 190/3663, Loss: 0.258409708738327
Epoch 1, 

In [14]:
# التقييم
model.eval()
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)  # نقل إلى الجهاز
        attention_mask = batch['attention_mask'].to(device)  # نقل إلى الجهاز
        labels = batch['labels'].to(device)  # نقل إلى الجهاز

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        print(f"Test Loss: {loss.item()}")


Test Loss: 0.34796154499053955
Test Loss: 0.4833264946937561
Test Loss: 0.5636488795280457
Test Loss: 0.6130918860435486
Test Loss: 0.7186622619628906
Test Loss: 0.40657296776771545
Test Loss: 0.46746253967285156
Test Loss: 0.4608495831489563
Test Loss: 0.47625023126602173
Test Loss: 0.40860265493392944
Test Loss: 0.7513885498046875
Test Loss: 0.5615928769111633
Test Loss: 0.5599784255027771
Test Loss: 0.6482925415039062
Test Loss: 0.4392029047012329
Test Loss: 0.5391635894775391
Test Loss: 0.7160273194313049
Test Loss: 0.6994509100914001
Test Loss: 0.6323190331459045
Test Loss: 0.5901555418968201
Test Loss: 0.3513769805431366
Test Loss: 0.42245328426361084
Test Loss: 0.5726600289344788
Test Loss: 0.44787439703941345
Test Loss: 0.4602750241756439
Test Loss: 0.8008947372436523
Test Loss: 0.6871098279953003
Test Loss: 0.3674945831298828
Test Loss: 0.5260723829269409
Test Loss: 0.42483362555503845
Test Loss: 0.6359506249427795
Test Loss: 0.5110348463058472
Test Loss: 0.4919239580631256
Te

In [ ]:
# التنبؤ
def generate_answer(question):
    inputs = tokenizer(question, return_tensors="pt").to(device)  # نقل إلى الجهاز
    outputs = model.generate(inputs['input_ids'], max_length=50)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# مثال استخدام
question = "انا اليوم اشعر بشعور كئيب جدا "
answer = generate_answer(question)
print(answer)

من الأفضل المتابعة مع معالج نفسي للتشخيص والتدخل العلاجي من خلال جلسات العلاج النفسي لعلاج الأسباب النفسية الكامنة التي تؤدي إلى الإكتئاب


In [17]:
# التنبؤ
def generate_answer(question):
    inputs = tokenizer(question, return_tensors="pt").to(device)  # نقل إلى الجهاز
    outputs = model.generate(inputs['input_ids'], max_length=50)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# مثال استخدام
question = "مرحبا انا اشعر بالقلق"
answer = generate_answer(question)
print(answer)

أنا متأكد من أنك ستبلي بلاء حسنا أنا متأكد من أنك ستبلي بلاء حسنا
